## Importing Modules

In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append('/home/junho/script')
import scjp

/home/junho/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# useful imports
import numpy as np
import scipy as scipy
import scanpy as sc
import scanpy.external as sce
import pandas as pd
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', None)
import pickle as pkl
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10,8]
import seaborn as sns
from collections import defaultdict, Counter
import networkx as nx
import igraph, re, glob
from bbknn import bbknn
from geosketch import gs
import scrublet as scr
import joblib as jl
from datetime import datetime
def timestamp():
    return datetime.now().strftime("%y%m%d%H%M")
import logging
import scipy.stats
from SCCAF import *

In [3]:
# setting scanpy
%matplotlib inline
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100, color_map='OrRd')
sc.logging.print_version_and_date()

Running Scanpy 1.7.2, on 2021-12-29 15:14.


In [4]:
nb_name = 'CART07.v02.HCA_Cancer_Geosketch_211116'
version = '.'.join(nb_name.split('.')[:2])+'.'
data = '_'.join(nb_name.split('.')[-1:][0].split('_')[:-2])
print('Version:', version)
print('Data:', data)

Version: CART07.v02.
Data: HCA_Cancer


In [5]:
base_folder = '/home/junho/links/single_cell_rnaseq/'

## V02: HCA+Tumor 합치고, 같이 geosketch + Tumor cell만 추가

## Loading Adata

In [6]:
hca = sc.read('%shealthy_control/healthy_merged_2021/CART05.v01.hca_210819.h5ad'%(base_folder))

Only considering the two last: ['.hca_210819', '.h5ad'].
Only considering the two last: ['.hca_210819', '.h5ad'].


In [7]:
hca

AnnData object with n_obs × n_vars = 590120 × 5697
    obs: 'Author', 'Organ', 'Organ_Specific', 'CellType_original', 'Batch', 'CellType', 'Method'
    var: 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Author_colors', 'CellType_singler_colors', 'Organ_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'X_umap_original'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [8]:
hca.obs

,Author,Organ,Organ_Specific,CellType_original,Batch,CellType,Method
LungTranscriptome7239220_LungTranscriptome7239220ACGGAGATCAAGGTAA-0,vieira19_Alveoli_and_parenchyma_anonymised,Lung,Lung_parenchyma,Ciliated,298C,Epithelial_cells,10X_3
LungTranscriptome7135919_GGCGTGTCATACCATG-0,vieira19_Alveoli_and_parenchyma_anonymised,Lung,Lung_parenchyma,Ciliated,290B,HSCs,10X_3
LungTranscriptome7135920_GCGACCATCAGTTAGC-0,vieira19_Alveoli_and_parenchyma_anonymised,Lung,Lung_parenchyma,Basal,290B,Epithelial_cells,10X_3
LungTranscriptome7239219_LungTranscriptome7239219CATTCGCAGCCGCCTA-0,vieira19_Alveoli_and_parenchyma_anonymised,Lung,Lung_parenchyma,Ciliated,298C,Epithelial_cells,10X_3
LungTranscriptome7135920_TCAACGAGTATCTGCA-0,vieira19_Alveoli_and_parenchyma_anonymised,Lung,Lung_parenchyma,Ciliated,290B,Epithelial_cells,10X_3
...,...,...,...,...,...,...,...
GSM4909273_N-MH275-Epi_TTTGTCACAGATGAGC-1-68,pal20_breast,Breast,Breast,ML,Patient_0275,Epithelial_cells,10X_3
GSM4909273_N-MH275-Epi_TTTGTCACATGCAACT-1-68,pal20_breast,Breast,Breast,LP,Patient_0275,Epithelial_cells,10X_3
GSM4909273_N-MH275-Epi_TTTGTCAGTAGGGACT-1-68,pal20_breast,Breast,Breast,LP,Patient_0275,Epithelial_cells,10X_3
GSM4909273_N-MH275-Epi_TTTGTCAGTATTCGTG-1-68,pal20_breast,Breast,Breast,ML,Patient_0275,Epithelial_cells,10X_3


### Filter out Spleen, Esophagus Data d/t LQ dataset

In [9]:
madisoon = [x for x in set(hca.obs['Author']) if 'madissoon20' in x]
madisoon

['madissoon20_oesophagus', 'madissoon20_spleen']

In [10]:
hca = hca[~hca.obs['Author'].isin(madisoon)]

/home/junho/anaconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [11]:
hca

View of AnnData object with n_obs × n_vars = 407916 × 5697
    obs: 'Author', 'Organ', 'Organ_Specific', 'CellType_original', 'Batch', 'CellType', 'Method'
    var: 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Author_colors', 'CellType_singler_colors', 'Organ_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'X_umap_original'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [12]:
hca.obs['Author,Organ,Organ_Specific,Method'.split(',')].drop_duplicates().sort_values('Organ')

,Author,Organ,Organ_Specific,Method
GSM4909268_N-MH0023-Total_AAACCTGAGAACAATC-1-0,pal20_breast,Breast,Breast,10X_3
AAGGAGCAGTTAGGTA_Colon-1_Enterocyte,wang20_colon,Colon,Colon,10X_3
AAACGGGAGGTGCAAC-1-Pan_T7935487,james20,Colon,Colon_immune_cells,10X_5
AAACCTGCAAGGCTCC-1-4861STDY7208411,james20,Colon,Colon_immune_cells,10X_3
ftCSMC01N__c00005,HQ.Dinh,Fallopian_tube,Fallopian_tube,10X_3
LA_1600_1_ACCCACTGTCGGCACT-1,tucker20_heart,Heart,Heart,10X_3
AAATGCCCAATCTGCA_Ileum-1_Enterocyte,wang20_ileum,Ileum,Ileum,10X_3
CZIKidney7632804_GCATGTATCCGAAGAG-1,stewart19_adult,Kidney,Kidney,10X_3
P1TLH_AAACCTGAGCAGCCTC_1,macparland18,Liver,Liver,10X_3
AAACCTGGTTGTGGCC-1-HCATisStab7509734,madissoon19_lung,Lung,Lung_parenchyma,10X_3


## Refining CellType annotation

In [14]:
set(hca.obs['CellType'])

{'Adipocytes',
 'Astrocytes',
 'B-cells',
 'BM_&_Progs',
 'B_cells',
 'CD4+_T-cells',
 'CD8+_T-cells',
 'CMPs',
 'Chondrocytes',
 'DCs',
 'Embryonic_stem_cells',
 'Endothelial_cells',
 'Eosinophils',
 'Epithelial_cells',
 'Erythroblasts',
 'Erythrocytes',
 'Fibroblasts',
 'GMPs',
 'Gametocytes',
 'HSC_-G-CSFs',
 'HSCs',
 'Hepatocytes',
 'Keratinocytes',
 'Macrophages',
 'Melanocytes',
 'Mesangial_cells',
 'Monocytes',
 'Myelocytes',
 'Myocytes',
 'NK_cells',
 'Neuroepithelial_cells',
 'Neurons',
 'Neutrophils',
 'Osteoblasts',
 'Pericytes',
 'Platelets',
 'Pro-B_cell_CD34+s',
 'Skeletal_muscles',
 'Smooth_muscle_cells',
 'Smooth_muscles',
 'T_cells',
 'Tissue_stem_cells',
 'iPS_cells'}

In [15]:
newct = []
for f in hca.obs['CellType']:
    if f[-1]=='s':
        newct.append(f[:-1])
    else:
        newct.append(f)

In [16]:
hca.obs['CellType_new'] = newct

Trying to set attribute `.obs` of view, copying.


In [18]:
hca.obs['CellType_new'] = ['Dendritic_cell' if 'DC' in x else x for x in hca.obs['CellType_new']]

In [19]:
hca.obs['CellType_new'] = ['B_cell' if 'B-cell' in x else x for x in hca.obs['CellType_new']]

In [20]:
set(hca.obs['CellType_new'])

{'Adipocyte',
 'Astrocyte',
 'BM_&_Prog',
 'B_cell',
 'CD4+_T-cell',
 'CD8+_T-cell',
 'CMP',
 'Chondrocyte',
 'Dendritic_cell',
 'Embryonic_stem_cell',
 'Endothelial_cell',
 'Eosinophil',
 'Epithelial_cell',
 'Erythroblast',
 'Erythrocyte',
 'Fibroblast',
 'GMP',
 'Gametocyte',
 'HSC',
 'HSC_-G-CSF',
 'Hepatocyte',
 'Keratinocyte',
 'Macrophage',
 'Melanocyte',
 'Mesangial_cell',
 'Monocyte',
 'Myelocyte',
 'Myocyte',
 'NK_cell',
 'Neuroepithelial_cell',
 'Neuron',
 'Neutrophil',
 'Osteoblast',
 'Pericyte',
 'Platelet',
 'Pro-B_cell_CD34+',
 'Skeletal_muscle',
 'Smooth_muscle',
 'Smooth_muscle_cell',
 'T_cell',
 'Tissue_stem_cell',
 'iPS_cell'}

In [21]:
hca.obs['CellType'] = hca.obs['CellType_new'].copy()

In [22]:
del hca.obs['CellType_new']

In [25]:
hca.write('%shealthy_control/healthy_merged_2021/%s%s.h5ad'%(base_folder, version, 'hca_annorefine_211116'))

/home/junho/anaconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])
... storing 'CellType' as categorical


In [6]:
hca=sc.read('%shealthy_control/healthy_merged_2021/%s%s.h5ad'%(base_folder, version, 'hca_annorefine_211116'))

Only considering the two last: ['.hca_annorefine_211116', '.h5ad'].
Only considering the two last: ['.hca_annorefine_211116', '.h5ad'].


In [11]:
hca = hca[hca.obs['Organ']!='Testis']

/home/junho/anaconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [18]:
hca

View of AnnData object with n_obs × n_vars = 401717 × 5697
    obs: 'Author', 'Organ', 'Organ_Specific', 'CellType_original', 'Batch', 'CellType', 'Method'
    var: 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Author_colors', 'CellType_singler_colors', 'Organ_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'X_umap_original'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [12]:
Counter(hca.obs['Organ'])

Counter({'Lung': 69991,
         'Pancreas': 7714,
         'Prostate': 32921,
         'Colon': 45979,
         'Liver': 8444,
         'Kidney': 40268,
         'Retina': 24426,
         'Ileum': 5980,
         'Rectum': 3797,
         'Heart': 50000,
         'Fallopian_tube': 4715,
         'Breast': 107482})

In [19]:
{a:b for a,b in zip(hca.obs['Author'],hca.obs['Organ'])}

{'vieira19_Alveoli_and_parenchyma_anonymised': 'Lung',
 'madissoon19_lung': 'Lung',
 'baron16': 'Pancreas',
 'henry18_0': 'Prostate',
 'james20': 'Colon',
 'macparland18': 'Liver',
 'stewart19_adult': 'Kidney',
 'voigt19': 'Retina',
 'menon19': 'Retina',
 'wang20_colon': 'Colon',
 'wang20_ileum': 'Ileum',
 'wang20_rectum': 'Rectum',
 'tucker20_heart': 'Heart',
 'HQ.Dinh': 'Fallopian_tube',
 'pal20_breast': 'Breast'}

## Cancer Dataset Calling & Merging

In [13]:
tumor = sc.read('%scancer_h5ad/total_merged.h5ad'%(base_folder))

In [14]:
tumor

AnnData object with n_obs × n_vars = 528152 × 36601
    obs: 'CellType', 'Author', 'Organ', 'Organ_Specific', 'Batch', 'Method', 'name'

In [15]:
Counter(tumor.obs['Organ'])

Counter({'Liver_Cancer': 62295,
         'Ovary_Cancer': 41117,
         'Lung_Cancer': 164127,
         'Colon_Cancer': 75254,
         'Pancreas_Cancer': 83733,
         'Breast_Cancer': 101626})

In [17]:
{a:b for a,b in zip(tumor.obs['Author'],tumor.obs['Organ'])}

{'Ankur_Sharma': 'Liver_Cancer',
 'Lichun.Ma': 'Liver_Cancer',
 'OH.Lee': 'Colon_Cancer',
 'Junbin.Qian': 'Breast_Cancer',
 'Qian': 'Colon_Cancer',
 'di.Magliano.MP': 'Pancreas_Cancer',
 'Sun.BaoFa': 'Pancreas_Cancer',
 'Sunny.Z.Wu': 'Breast_Cancer'}

In [31]:
tum = tumor[tumor.obs['CellType']=='Tumor_cells']

/home/junho/anaconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [32]:
tum_index = list(tum.obs.index)

In [33]:
len(tum_index)

92335

In [35]:
len(np.unique(tum_index))

92335

## Merging HCA with Tumor Data

In [36]:
ad = {'Normal':hca,'Tumor':tumor}

In [42]:
merged = scjp.merge_matrix(ad,use_raw=True)

Normal
Tumor
936068
Author
Organ
Organ_Specific
CellType_original
Batch
CellType
Method
name


In [43]:
merged.obs

,Author,Organ,Organ_Specific,CellType_original,Batch,CellType,Method,name
LungTranscriptome7239220_LungTranscriptome7239220ACGGAGATCAAGGTAA-0,vieira19_Alveoli_and_parenchyma_anonymised,Lung,Lung_parenchyma,Ciliated,298C,Epithelial_cell,10X_3,Normal
LungTranscriptome7135919_GGCGTGTCATACCATG-0,vieira19_Alveoli_and_parenchyma_anonymised,Lung,Lung_parenchyma,Ciliated,290B,HSC,10X_3,Normal
LungTranscriptome7135920_GCGACCATCAGTTAGC-0,vieira19_Alveoli_and_parenchyma_anonymised,Lung,Lung_parenchyma,Basal,290B,Epithelial_cell,10X_3,Normal
LungTranscriptome7239219_LungTranscriptome7239219CATTCGCAGCCGCCTA-0,vieira19_Alveoli_and_parenchyma_anonymised,Lung,Lung_parenchyma,Ciliated,298C,Epithelial_cell,10X_3,Normal
LungTranscriptome7135920_TCAACGAGTATCTGCA-0,vieira19_Alveoli_and_parenchyma_anonymised,Lung,Lung_parenchyma,Ciliated,290B,Epithelial_cell,10X_3,Normal
...,...,...,...,...,...,...,...,...
bcCID4398N__c04444,Sunny.Z.Wu,Breast_Cancer,Breast,n/a,bcCID4398N,Dendritic_cell,10X_3,Tumor
bcCID4398N__c04445,Sunny.Z.Wu,Breast_Cancer,Breast,n/a,bcCID4398N,Dendritic_cell,10X_3,Tumor
bcCID4398N__c04447,Sunny.Z.Wu,Breast_Cancer,Breast,n/a,bcCID4398N,Dendritic_cell,10X_3,Tumor
bcCID4398N__c04450,Sunny.Z.Wu,Breast_Cancer,Breast,n/a,bcCID4398N,Dendritic_cell,10X_3,Tumor


In [44]:
adata = merged.copy()

In [45]:
scjp.jhk.ad_summary(adata)

         max      mean  min  len_obs  len_var
0  21.731197  0.070502  0.0   936068    36601


In [46]:
adata.raw = adata

In [47]:
adata

AnnData object with n_obs × n_vars = 936068 × 36601
    obs: 'Author', 'Organ', 'Organ_Specific', 'CellType_original', 'Batch', 'CellType', 'Method', 'name'

In [48]:
len(set(adata.obs['Author']))

24

In [50]:
Counter(sorted(adata.obs['CellType']))

Counter({'Adipocyte': 59249,
         'Astrocyte': 6753,
         'BM_&_Prog': 1,
         'B_cell': 56442,
         'CD4+_T-cell': 56759,
         'CD8+_T-cell': 105797,
         'CMP': 1,
         'Chondrocyte': 965,
         'Dendritic_cell': 2971,
         'Embryonic_stem_cell': 42,
         'Embryonic_stem_cells': 1,
         'Endothelial_cell': 35895,
         'Eosinophil': 25,
         'Epithelial_cell': 155094,
         'Erythroblast': 6,
         'Erythrocyte': 4366,
         'Fibroblast': 69596,
         'GMP': 4,
         'Gametocyte': 1141,
         'HSC': 12384,
         'HSC_-G-CSF': 2,
         'Hepatocyte': 17734,
         'Keratinocyte': 13093,
         'MSC': 1,
         'Macrophage': 80411,
         'Melanocyte': 1188,
         'Mesangial_cell': 8300,
         'Monocyte': 59721,
         'Muscle_cell': 485,
         'Myelocyte': 4,
         'Myocyte': 2185,
         'NK_cell': 40621,
         'Neuroepithelial_cell': 538,
         'Neuron': 19559,
         'Neutrophil

In [51]:
num = dict(Counter(adata.obs['Batch']))

In [52]:
for f in num.items():
    if f[1]<3:
        print(f[0])

## Geosketch HCA Data

In [54]:
sketch_index = scjp.get_sketch(adata,'Author',folds=10,how='pd',min_num_per_key=10000)

Sun.BaoFa
from: 50151


/home/junho/anaconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
extracting highly variable genes
    finished (0:00:00)
/home/junho/anaconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
... as `zero_center=True`, sparse input is densified and may lead to large memory consumption


removing cc_genes...


computing PCA
    with n_comps=50
    finished (0:00:13)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:17)
computing Diffusion Maps using n_comps=15(=n_dcs)
computing transitions
    finished (0:00:00)
    eigenvalues of transition matrix
    [1.         0.99941355 0.9988128  0.99855596 0.998423   0.998305
     0.99820524 0.9981662  0.99779123 0.9975768  0.9968231  0.9965378
     0.99561113 0.99545807 0.9952841 ]
    finished: added
    'X_diffmap', diffmap coordinates (adata.obsm)
    'diffmap_evals', eigenvalues of transition matrix (adata.uns) (0:00:02)


to select: 10000
length of sketch: 17068
wang20_colon
from: 4329


...

to select: 10086
length of sketch: 300505
vieira19_Alveoli_and_parenchyma_anonymised
from: 12971


/home/junho/anaconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
extracting highly variable genes
    finished (0:00:00)
/home/junho/anaconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
... as `zero_center=True`, sparse input is densified and may lead to large memory consumption


removing cc_genes...


computing PCA
    with n_comps=50
    finished (0:00:02)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:01)
computing Diffusion Maps using n_comps=15(=n_dcs)
computing transitions
    finished (0:00:00)
    eigenvalues of transition matrix
    [1.         0.99987674 0.99958134 0.9989163  0.9988016  0.9980949
     0.9976477  0.99720824 0.9961371  0.9957371  0.99536115 0.9951033
     0.9943981  0.9939535  0.99378866]
    finished: added
    'X_diffmap', diffmap coordinates (adata.obsm)
    'diffmap_evals', eigenvalues of transition matrix (adata.uns) (0:00:01)


to select: 10000
length of sketch: 312485


In [60]:
len(sketch_index)

312485

In [56]:
to_keep=tum_index+sketch_index

In [57]:
len(to_keep)

404820

In [61]:
to_keep=list(np.unique(to_keep))

In [63]:
adsketch = adata[adata.obs.index.isin(to_keep)]

/home/junho/anaconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [64]:
adsketch

View of AnnData object with n_obs × n_vars = 374005 × 36601
    obs: 'Author', 'Organ', 'Organ_Specific', 'CellType_original', 'Batch', 'CellType', 'Method', 'name'

In [65]:
Counter(adsketch.obs['CellType'])

Counter({'Epithelial_cell': 58531,
         'HSC': 5618,
         'Monocyte': 20674,
         'Erythrocyte': 2089,
         'Adipocyte': 23938,
         'Macrophage': 19075,
         'Dendritic_cell': 1436,
         'B_cell': 17500,
         'CD8+_T-cell': 18873,
         'NK_cell': 9001,
         'Endothelial_cell': 17218,
         'Fibroblast': 30644,
         'Skeletal_muscle': 8944,
         'CD4+_T-cell': 10060,
         'T_cell': 38,
         'Keratinocyte': 1910,
         'Astrocyte': 4591,
         'Myocyte': 969,
         'Chondrocyte': 408,
         'Mesangial_cell': 3178,
         'Smooth_muscle': 138,
         'Smooth_muscle_cell': 6,
         'Pericyte': 652,
         'Neuron': 13763,
         'iPS_cell': 86,
         'Hepatocyte': 8691,
         'Neuroepithelial_cell': 341,
         'Tissue_stem_cell': 24,
         'Eosinophil': 17,
         'Gametocyte': 1140,
         'Embryonic_stem_cell': 25,
         'Melanocyte': 1042,
         'Erythroblast': 5,
         'Platelet'

In [66]:
Counter(adsketch.obs['Author'])

Counter({'vieira19_Alveoli_and_parenchyma_anonymised': 11980,
         'madissoon19_lung': 16678,
         'baron16': 7714,
         'guo18_donor': 6199,
         'henry18_0': 14630,
         'james20': 15775,
         'macparland18': 8444,
         'stewart19_adult': 14216,
         'voigt19': 4335,
         'menon19': 13708,
         'wang20_colon': 4329,
         'wang20_ileum': 5980,
         'wang20_rectum': 3797,
         'tucker20_heart': 16283,
         'HQ.Dinh': 4715,
         'pal20_breast': 18149,
         'Ankur_Sharma': 18966,
         'Lichun.Ma': 17915,
         'OH.Lee': 50569,
         'Junbin.Qian': 25029,
         'Qian': 16009,
         'di.Magliano.MP': 18694,
         'Sun.BaoFa': 26797,
         'Sunny.Z.Wu': 33094})

In [67]:
Counter(adsketch.obs['Organ'])

Counter({'Lung': 28658,
         'Pancreas': 7714,
         'Testis': 6199,
         'Prostate': 14630,
         'Colon': 20104,
         'Liver': 8444,
         'Kidney': 14216,
         'Retina': 18043,
         'Ileum': 5980,
         'Rectum': 3797,
         'Heart': 16283,
         'Fallopian_tube': 4715,
         'Breast': 18149,
         'Liver_Cancer': 36881,
         'Ovary_Cancer': 15593,
         'Lung_Cancer': 32957,
         'Colon_Cancer': 34804,
         'Pancreas_Cancer': 45491,
         'Breast_Cancer': 41347})

In [72]:
adsketch.write('%shca_cancer_merge/%s%s.h5ad'%(base_folder, version, 'hca_cancer_merge_sketch2_211116'))

/home/junho/anaconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1210: ImplicitModificationWarning: Initializing view as actual.
  "Initializing view as actual.", ImplicitModificationWarning
Trying to set attribute `.obs` of view, copying.
... storing 'Author' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Organ' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Organ_Specific' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'CellType_original' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Batch' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'CellType' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Method' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'name' as categorical
